Set up an Azure AI Search service and index basic data

In [25]:
! pip install azure-search-documents==11.6.0b1 --quiet
! pip install azure-identity --quiet
! pip install python-dotenv --quiet

search_endpoint: str = "https://kodeco-ai-search-service-paid-basic.search.windows.net"
search_api_key: str = "KdBoplAH20sep1dayhZDh34Uvg2K7S3UyTzqRyutD9AzSeDMLPmE"
index_name: str = "television-shows"

from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(search_api_key)

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex
)

# Create a search schema
index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=credential)
fields = [
        SimpleField(name="ShowId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="ShowName", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Genres", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),
        SimpleField(name="Seasons", type=SearchFieldDataType.Int32, facetable=True, filterable=True, sortable=True),
        SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Cast", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),
        SearchableField(name="StreamingPlatform", type=SearchFieldDataType.String, sortable=True),
        SimpleField(name="ReleaseDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True)
    ]

scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['Genres', 'Cast']}]

# Create the search index=
index = SearchIndex(name=index_name, fields=fields, suggesters=suggester, scoring_profiles=scoring_profiles)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 television-shows created


In [26]:
documents = [
    {
        "@search.action": "upload",
        "ShowId": "1",
        "ShowName": "Stranger Things",
        "Description": "A thrilling Netflix series set in the 1980s that combines horror, science fiction, and nostalgia.",
        "Category": "TV Series",
        "Genres": ["Horror", "Science Fiction", "Drama"],
        "Seasons": 4,
        "Rating": 4.7,
        "Cast": ["Millie Bobby Brown", "Finn Wolfhard", "Winona Ryder"],
        "StreamingPlatform": "Netflix",
        "ReleaseDate": "2016-07-15T00:00:00Z"
    },
    {
        "@search.action": "upload",
        "ShowId": "2",
        "ShowName": "The Mandalorian",
        "Description": "A Star Wars spin-off series following the journey of a lone bounty hunter in the outer reaches of the galaxy.",
        "Category": "TV Series",
        "Genres": ["Action", "Adventure", "Science Fiction"],
        "Seasons": 3,
        "Rating": 4.8,
        "Cast": ["Pedro Pascal", "Gina Carano", "Carl Weathers"],
        "StreamingPlatform": "Disney+",
        "ReleaseDate": "2019-11-12T00:00:00Z"
    },
    {
        "@search.action": "upload",
        "ShowId": "3",
        "ShowName": "The Crown",
        "Description": "A historical drama series chronicling the reign of Queen Elizabeth II and the events that shaped the second half of the 20th century.",
        "Category": "TV Series",
        "Genres": ["Drama", "History"],
        "Seasons": 6,
        "Rating": 4.5,
        "Cast": ["Claire Foy", "Olivia Colman", "Imelda Staunton"],
        "StreamingPlatform": "Netflix",
        "ReleaseDate": "2016-11-04T00:00:00Z"
    },
    {
        "@search.action": "upload",
        "ShowId": "4",
        "ShowName": "Breaking Bad",
        "Description": "A high school chemistry teacher turned methamphetamine manufacturer partners with a former student to build a drug empire.",
        "Category": "TV Series",
        "Genres": ["Crime", "Drama", "Thriller"],
        "Seasons": 5,
        "Rating": 4.9,
        "Cast": ["Bryan Cranston", "Aaron Paul", "Anna Gunn"],
        "StreamingPlatform": "AMC",
        "ReleaseDate": "2008-01-20T00:00:00Z"
    },
    {
        "@search.action": "upload",
        "ShowId": "5",
        "ShowName": "Friends",
        "Description": "A beloved sitcom that follows the lives of six friends living in New York City as they navigate work, relationships, and life in general.",
        "Category": "TV Series",
        "Genres": ["Comedy", "Romance"],
        "Seasons": 10,
        "Rating": 4.7,
        "Cast": ["Jennifer Aniston", "Courteney Cox", "Matthew Perry"],
        "StreamingPlatform": "HBO Max",
        "ReleaseDate": "1994-09-22T00:00:00Z"
    },
    {
        "@search.action": "upload",
        "ShowId": "6",
        "ShowName": "Game of Thrones",
        "Description": "An epic fantasy series based on the novels by George R.R. Martin, depicting the struggle for the Iron Throne in the Seven Kingdoms of Westeros.",
        "Category": "TV Series",
        "Genres": ["Fantasy", "Drama", "Adventure"],
        "Seasons": 8,
        "Rating": 4.6,
        "Cast": ["Emilia Clarke", "Kit Harington", "Peter Dinklage"],
        "StreamingPlatform": "HBO",
        "ReleaseDate": "2011-04-17T00:00:00Z"
    }
]


In [27]:
search_client = SearchClient(endpoint=search_endpoint,
                      index_name=index_name,
                      credential=credential)
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

    index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=credential)

Upload of new document succeeded: True


In [28]:
# Run an empty query (returns selected fields, all documents)
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='ShowName,Description',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["ShowName"])
    print(f"Description: {result['Description']}")

Total Documents Matching Query: 0


In [29]:
# Run a term query
results =  search_client.search(query_type='simple',
    search_text="drama" ,
    select='ShowName,Description,Genres',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["ShowName"])
    print(f"Description: {result['Description']}")

Total Documents Matching Query: 0


In [30]:
# try:
#     result = index_client.delete_index(index_name)
#     print ('Index', index_name, 'Deleted')
# except Exception as ex:
#     print (ex)

In [31]:
# try:
#     result = index_client.get_index(index_name)
#     print (result)
# except Exception as ex:
#     print (ex)

: 